In [1]:
import numpy as np
import mido
import tensorflow as tf
import time

from keras.models import load_model
from mido import MidiFile, MidiTrack, Message
from random import randrange


model = load_model('sb.h5')

Using TensorFlow backend.


In [2]:
note_one = 60
note_two = 65
note_three = 67
note_four = 67

In [9]:
#Midi Notes range from 0 - 127
#Input 4 notes for a chord progression
user_input = [[[note_one/11], [note_two/11], [note_three/11], [note_four/11]]]
user_input = np.array(user_input)
#reshaping input to 3D
user_input.reshape(1,1,4)

array([[[5.45454545, 5.90909091, 6.09090909, 6.09090909]]])

In [10]:
#Performing inference on the model
start_time = time.time()
predict = model.predict_on_batch(user_input)
elapsed_time = time.time() - start_time
#Adjusting from normalization
predict = predict * 11
predict = abs(predict)
predict.astype(int)

array([[[ 5],
        [ 9],
        [13],
        [16],
        [18],
        [20],
        [22],
        [23],
        [24],
        [25],
        [25],
        [25],
        [26],
        [26],
        [26],
        [26]]])

In [11]:
elapsed_time

0.03588461875915527

In [12]:
#helper function to classify notes into bins
def note_reg(note):
    C = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]
    C_sharp = [1, 13, 25, 37, 49, 61, 73, 85, 97, 109, 121]
    D = [2, 14, 26, 38, 50, 62, 74, 86, 98, 110, 122]
    D_sharp = [3, 15, 27, 39, 51, 63, 75, 87, 99, 111, 123]
    E = [4, 16, 28, 40, 52, 64, 76, 88, 100, 112, 124]
    F = [5, 17, 29, 41, 53, 65, 77, 89, 101, 113, 125]
    F_sharp = [6, 18, 30, 42, 54, 66, 78, 90, 102, 114, 126]
    G = [7, 19, 31, 43, 55, 67, 79, 91, 103, 115, 127]
    G_sharp = [8, 20, 32, 44, 56, 68, 80, 92, 104, 116]
    A = [9, 21, 33, 45, 57, 69, 81, 93, 105, 117]
    A_sharp = [10, 22, 34, 46, 58, 70, 82, 94, 106, 118]
    B = [11, 23, 35, 47, 59, 71, 83, 95, 107, 119]
    if (note in C):
        return 0
    elif(note in C_sharp):
        return 1
    elif(note in D):
        return 2
    elif(note in D_sharp):
        return 3
    elif(note in E):
        return 4
    elif(note in F):
        return 5
    elif(note in F_sharp):
        return 6
    elif(note in G):
        return 7
    elif(note in G_sharp):
        return 8
    elif(note in A):
        return 9
    elif(note in A_sharp):
        return 10
    elif(note in B):
        return 11
    else:
        return 0

In [13]:
#timing array
timing = np.load('csp.npy',allow_pickle=True)
avg = sum([note_one,note_two,note_three,note_four]) / len([note_one,note_two,note_three,note_four])

In [14]:
#new midi file
mid = MidiFile()
track2 = MidiTrack()
mid.tracks.append(track2)

track = MidiTrack()
mid.tracks.append(track)

#scalable octave variable
octave = 12 * int(avg /10)
#timing array index
t = randrange(len(timing))

track.append(Message('program_change', program=0, time=0))
track2.append(Message('program_change', program=0, time=0))
for x in range(4):
    track2.append(Message('note_on', note= note_one, velocity=127, time=0))
    track2.append(Message('note_off', note= note_one, velocity=127, time=1920))
    track2.append(Message('note_on', note= note_two, velocity=127, time=0))
    track2.append(Message('note_off', note= note_two, velocity=127, time=1920))
    track2.append(Message('note_on', note= note_three, velocity=127, time=0))
    track2.append(Message('note_off', note= note_three, velocity=127, time=1920))
    track2.append(Message('note_on', note= note_four, velocity=127, time=0))
    track2.append(Message('note_off', note= note_four, velocity=127, time=1920))
for x in range(5):
    for i in range(12):
        track.append(Message('note_on', note= octave + note_reg(int(predict[0][i][0])), velocity=64, time=20))
        track.append(Message('note_off', note= octave + note_reg(int(predict[0][i][0])), velocity=64, time=timing[t].get(str(i))))
mid.save('song.mid')